In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
graf = gpd.read_file(r"C:\Users\btok\DataspellProjects\AdvancedGIS\data1\PoliceData.gdb", driver='fileGDB', layer="GraffitiIncidents")
pz = gpd.read_file(r"C:\Users\btok\DataspellProjects\AdvancedGIS\data1\PoliceData.gdb", driver='fileGDB', layer="PatrolZones")

DriverError: E:\Dataspell Projects E\AdvancedGIS\data1\PoliceData.gdb: No such file or directory

In [ ]:
# join point to polygon, group by name and count and assign these values to patrol zone INCIDENTS column
pz["INCIDENTS"] = gpd.sjoin(graf, pz, how="inner", predicate="within").groupby(["NAME"]).size().reset_index(name="incidentsPerZone")["incidentsPerZone"]

In [ ]:
# calculate and assign priority to each zone
incRate = pz["INCIDENTS"] / (pz["SHAPE_Area"] / 2589988.11)
pz.loc[incRate > 15, "PRIORITY"] = "TOP CONCERN"    
pz.loc[(incRate < 15) & (incRate >= 12), "PRIORITY"] = "HIGH CONCERN"    
pz.loc[(incRate < 12) & (incRate >= 6), "PRIORITY"] = "MEDIUM CONCERN"    
pz.loc[incRate < 6, "PRIORITY"] = "LOW CONCERN"
pz

In [ ]:
pz["center"] = pz["geometry"].centroid
pz_points = pz.copy()
pz_points.set_geometry("center", inplace=True)

In [ ]:
ax = pz.plot(column="PRIORITY", figsize = (15, 12), cmap="Accent", edgecolor = "lightgrey", linewidth = 0.5)
texts = []

for x, y, label in zip(pz_points.geometry.x, pz_points.geometry.y, pz_points["PRIORITY"]):
    texts.append(plt.text(x, y, label, fontsize = 8))